# Moon Cycles Deep Search (Moon-Only Features)

This notebook is a **Moon-phase-only** research pipeline for Bitcoin direction prediction.

What this notebook does:
1. Loads market data from parquet.
2. Builds only Moon-phase features (Sun + Moon geometry, no other planets).
3. Tunes Gaussian label parameters.
4. Evaluates two protocols:
   - Classic split: `70 / 15 / 15`
   - Walk-forward split: `50% warm-up + 10/10/10/10/10 blocks`, where each block is `5% val + 5% test`
5. Runs full visual diagnostics **before training** (baselines) and **after training** (XGBoost).

Important:
- Everything expensive is cached.
- Split boundaries are shown on charts, so train/validation/test are fully transparent.

## Speed And Caching Strategy

Why this notebook runs much faster than a naive implementation:
- We compute astro positions only for **Sun and Moon**.
- We cache market slices, Moon features, labels, merged datasets, and whole experiment runs.
- We reuse the same Moon feature table while sweeping Gaussian parameters.
- We keep heavy logic inside reusable Python modules instead of repeating it in notebook cells.

This means first run can be heavy, but repeated runs should be much faster.

In [ ]:
from pathlib import Path
import sys

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import display

PROJECT_ROOT = Path('/home/rut/ostrofun')
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

from RESEARCH2.Moon_cycles.moon_data import (
    MoonLabelConfig,
    build_moon_dataset_for_gauss,
    build_moon_phase_features,
    load_market_slice,
)
from RESEARCH2.Moon_cycles.search_utils import (
    WalkForwardConfig,
    XgbConfig,
    run_gauss_search,
)
from RESEARCH2.Moon_cycles.eval_utils import compute_binary_metrics
from RESEARCH2.Moon_cycles.eval_visuals import VisualizationConfig, evaluate_with_visuals

pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 160)

In [ ]:
# ------------------------------
# Research configuration block
# ------------------------------
# We keep all knobs in one place so experiments are reproducible.

START_DATE = '2017-11-01'
END_DATE = None
USE_CACHE = True
VERBOSE = True

# Label parameters that stay fixed while we tune Gaussian window/std.
LABEL_CFG = MoonLabelConfig(
    horizon=1,
    move_share=0.5,
    label_mode='balanced_detrended',
    price_mode='raw',
)

# Gaussian grid to tune.
GAUSS_WINDOWS = [101, 151, 201, 251, 301]
GAUSS_STDS = [30.0, 50.0, 70.0, 90.0]

# Model hyperparameters.
MODEL_CFG = XgbConfig(
    n_estimators=500,
    max_depth=6,
    learning_rate=0.03,
    colsample_bytree=0.8,
    subsample=0.8,
    early_stopping_rounds=50,
)

# Walk-forward split protocol.
WF_CFG = WalkForwardConfig(
    warmup_ratio=0.50,
    block_ratios=(0.10, 0.10, 0.10, 0.10, 0.10),
    val_fraction_inside_block=0.50,
)

# Visual defaults: highly visual by design.
VIS_CFG = VisualizationConfig(
    rolling_window_days=90,
    rolling_min_periods=30,
)

print('Config loaded.')
print('Gaussian grid size =', len(GAUSS_WINDOWS) * len(GAUSS_STDS))

In [ ]:
# -------------------------------------------
# Load market data and build Moon-only features
# -------------------------------------------
# This is the main expensive astro stage.
# On repeated runs it should come from cache.

df_market = load_market_slice(
    start_date=START_DATE,
    end_date=END_DATE,
    use_cache=USE_CACHE,
    verbose=VERBOSE,
)

df_moon_features = build_moon_phase_features(
    df_market=df_market,
    use_cache=USE_CACHE,
    verbose=VERBOSE,
    progress=True,
)

print('Market rows:', len(df_market))
print('Moon feature rows:', len(df_moon_features))
print('Market range:', df_market['date'].min().date(), '->', df_market['date'].max().date())

display(df_moon_features.head(5))

## Protocol A: Classic 70/15/15

We tune Gaussian parameters and evaluate each candidate using:
- Train on first 70%
- Tune threshold on next 15%
- Report final quality on last 15%

In [ ]:
classic_search = run_gauss_search(
    df_market=df_market,
    df_moon_features=df_moon_features,
    gauss_windows=GAUSS_WINDOWS,
    gauss_stds=GAUSS_STDS,
    label_cfg=LABEL_CFG,
    model_cfg=MODEL_CFG,
    wf_cfg=WF_CFG,
    protocol='classic',
    use_cache=USE_CACHE,
    verbose=VERBOSE,
)

classic_table = classic_search['results_table']
print('Classic search completed. Top rows:')
display(classic_table.head(10))

In [ ]:
# -------------------------------------------------------
# Full evaluation for the best classic configuration
# -------------------------------------------------------
# We evaluate BEFORE training (majority baseline), then AFTER training.

classic_best = classic_search['best_result']
classic_best_row = classic_search['best_row']
print('Best classic config:')
display(pd.DataFrame([classic_best_row]))

classic_pred = classic_best['predictions'].copy()
classic_test = classic_pred[classic_pred['split_role'] == 'test'].copy().reset_index(drop=True)
classic_test = classic_test.dropna(subset=['pred_label'])

print('Classic test rows:', len(classic_test))

# BEFORE training: majority baseline.
classic_before = evaluate_with_visuals(
    df_plot=classic_test,
    y_true=classic_test['target'].to_numpy(dtype=np.int32),
    y_pred=classic_test['baseline_majority'].to_numpy(dtype=np.int32),
    y_prob_up=None,
    title='Classic protocol - BEFORE training (majority baseline)',
    vis_cfg=VIS_CFG,
    show_visuals=True,
)

# AFTER training: tuned XGBoost predictions.
classic_after = evaluate_with_visuals(
    df_plot=classic_test,
    y_true=classic_test['target'].to_numpy(dtype=np.int32),
    y_pred=classic_test['pred_label'].to_numpy(dtype=np.int32),
    y_prob_up=classic_test['pred_proba_up'].to_numpy(dtype=float),
    title='Classic protocol - AFTER training (Moon-only model)',
    vis_cfg=VIS_CFG,
    show_visuals=True,
)

# Extra number-only baseline check: random baseline with train class prior.
classic_random_metrics = compute_binary_metrics(
    y_true=classic_test['target'].to_numpy(dtype=np.int32),
    y_pred=classic_test['baseline_random'].to_numpy(dtype=np.int32),
)
print('Classic random baseline metrics:', classic_random_metrics)

## Protocol B: Walk-Forward (50% warm-up + 10% blocks)

We now use an expanding-window protocol closer to real deployment:
- First 50% is initial training history.
- Remaining history is split into five 10% sequential blocks.
- Each 10% block is split into 5% validation + 5% test.

In [ ]:
walk_search = run_gauss_search(
    df_market=df_market,
    df_moon_features=df_moon_features,
    gauss_windows=GAUSS_WINDOWS,
    gauss_stds=GAUSS_STDS,
    label_cfg=LABEL_CFG,
    model_cfg=MODEL_CFG,
    wf_cfg=WF_CFG,
    protocol='walk_forward',
    use_cache=USE_CACHE,
    verbose=VERBOSE,
)

walk_table = walk_search['results_table']
print('Walk-forward search completed. Top rows:')
display(walk_table.head(10))

In [ ]:
# -----------------------------------------------------------
# Full evaluation for the best walk-forward configuration
# -----------------------------------------------------------
# We evaluate on concatenated test blocks from all folds.

walk_best = walk_search['best_result']
walk_best_row = walk_search['best_row']
print('Best walk-forward config:')
display(pd.DataFrame([walk_best_row]))

walk_pred = walk_best['predictions'].copy()
walk_test = walk_pred[walk_pred['split_role'] == 'test'].copy().reset_index(drop=True)
walk_test = walk_test.dropna(subset=['pred_label'])

print('Walk-forward concatenated test rows:', len(walk_test))

# BEFORE training: majority baseline per fold (already stored in merged test stream).
walk_before = evaluate_with_visuals(
    df_plot=walk_test,
    y_true=walk_test['target'].to_numpy(dtype=np.int32),
    y_pred=walk_test['baseline_majority'].to_numpy(dtype=np.int32),
    y_prob_up=None,
    title='Walk-forward - BEFORE training (majority baseline)',
    vis_cfg=VIS_CFG,
    show_visuals=True,
)

# AFTER training: model predictions from each fold test block.
walk_after = evaluate_with_visuals(
    df_plot=walk_test,
    y_true=walk_test['target'].to_numpy(dtype=np.int32),
    y_pred=walk_test['pred_label'].to_numpy(dtype=np.int32),
    y_prob_up=walk_test['pred_proba_up'].to_numpy(dtype=float),
    title='Walk-forward - AFTER training (Moon-only model)',
    vis_cfg=VIS_CFG,
    show_visuals=True,
)

walk_random_metrics = compute_binary_metrics(
    y_true=walk_test['target'].to_numpy(dtype=np.int32),
    y_pred=walk_test['baseline_random'].to_numpy(dtype=np.int32),
)
print('Walk-forward random baseline metrics:', walk_random_metrics)

if 'fold_table' in walk_best:
    print('Per-fold quality table:')
    display(walk_best['fold_table'])

In [ ]:
# -------------------------------------------
# Compare best protocol summaries side by side
# -------------------------------------------

comparison = pd.DataFrame([
    {
        'protocol': 'classic_70_15_15',
        **classic_search['best_row'],
    },
    {
        'protocol': 'walk_forward',
        **walk_search['best_row'],
    },
])

cols_order = [
    'protocol',
    'gauss_window',
    'gauss_std',
    'test_acc',
    'test_bal_acc',
    'test_mcc',
    'test_recall_min',
    'test_recall_down',
    'test_recall_up',
    'baseline_majority_test_acc',
    'baseline_random_test_acc',
    'p_value_vs_random',
]

print('Best candidates comparison:')
display(comparison[cols_order])

In [ ]:
# ------------------------------------------------------
# Visual heatmaps: how Gaussian parameters affect quality
# ------------------------------------------------------

def plot_gauss_heatmap(results_df: pd.DataFrame, metric: str, title: str) -> None:
    pivot = results_df.pivot_table(index='gauss_window', columns='gauss_std', values=metric)
    plt.figure(figsize=(8, 5))
    sns.heatmap(pivot, annot=True, fmt='.3f', cmap='viridis')
    plt.title(title)
    plt.xlabel('gauss_std')
    plt.ylabel('gauss_window')
    plt.tight_layout()
    plt.show()

plot_gauss_heatmap(classic_table, 'test_recall_min', 'Classic: Recall_MIN by Gaussian params')
plot_gauss_heatmap(classic_table, 'test_mcc', 'Classic: MCC by Gaussian params')

plot_gauss_heatmap(walk_table, 'test_recall_min', 'Walk-forward: Recall_MIN by Gaussian params')
plot_gauss_heatmap(walk_table, 'test_mcc', 'Walk-forward: MCC by Gaussian params')

## Final Notes

- If `p_value_vs_random` is very small, model accuracy is unlikely to be random luck.
- Always compare against both baselines (majority and random).
- For production, use the best Gaussian config from this notebook and retrain on all available data right before live forecasting.